# 数据聚合与分组运算

+ 根据一个或多个键（可以是函数、数组或DataFrame列名）拆分pandas对象
+ 计算分组统计摘要，如计数、平均值、标准差，或用户自定义函数
+ 对DataFrame的列应用各种各样的函数
+ 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等
+ 计算透视表或交叉表
+ 执行分位数分析以及其他分组分析

## 目录
+ GroupBy技术
 + 根据DataFrame某个列分组
 + 根据与待分组的轴等长的列表或数组进行分组
 + 得到分组后的内容：对分组进行迭代、字典
 + 在GroupBy对象中选取一个或一组列
 + 通过字典或Series进行分组
 + 通过函数进行分组
 + 根据索引级别分组


+ 数据分组聚合运算
 + 聚合运算的种类
     + 经过优化的Groupby的方法
     + 分组对象上已经定义好的方法
     + 自己定义的聚合函数
     + 某些非聚合运算
 + 面向列的多函数应用
     + 对一个列应用多个函数
     + 为返回的DataFrame的列命名
     + 对多个列应用多个函数
     + 对不同的列应用不同的函数
 + 以“无索引”的形式返回聚合数据


+ 分组级运算和转换
	+ 先聚合再合并
	+ transform方法
	+ apply: 一般性的“拆分-应用-合并”
		+ 禁止分组键
	+ 分位数和桶分析
		+ 桶分析
		+ 分位数
        
        
+ 透视表和交叉表
	+ 透视表
	+ 交叉表

In [1]:
from numpy.random import randn
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd

## GroupBy技术

In [2]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.508505,0.367653,a,one
1,0.494701,0.250424,a,two
2,-1.082100,-1.702437,b,one
3,0.301170,-0.971386,b,two
4,-1.171429,0.292727,a,one


### 根据DataFrame某个列分组

In [3]:
grouped = df['data1'].groupby(df['key1']) # 产生一个GroupBy对象，该对象拥有接下来对各分组执行运算所需的一切信息
grouped

In [4]:
grouped.mean() # 计算分组平均值

key1
a   -0.395077
b   -0.390465
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean() # 通过两个键对数据分组，得到的Series具有一个层次化索引
means

key1  key2
a     one    -0.839967
      two     0.494701
b     one    -1.082100
      two     0.301170
Name: data1, dtype: float64

In [6]:
means.unstack() # 层次化索引的unstack方法

key2,one,two
key1,,
a,-0.839967,0.494701
b,-1.082100,0.301170


In [7]:
df.groupby('key1').mean() # 对整个df进行聚合，默认情况下，所有数值列都会被聚合（非数值列俗称“麻烦列”，默认不聚合）

,data1,data2
key1,,
a,-0.395077,0.303601
b,-0.390465,-1.336912


In [8]:
df.groupby(['key1', 'key2']).mean() # 直接用列名做分组键（可以是字符串、数字或其他Python对象）

data1 
 data2 
 
 
 key1 
 key2 
 
 
 
 
 
 
 a 
 one 
 -0.839967 
 0.330190 
 
 
 two 
 0.494701 
 0.250424 
 
 
 b 
 one 
 -1.082100 
 -1.702437 
 
 
 two 
 0.301170 
 -0.971386

In [9]:
df.groupby(['key1', 'key2']).size() # size方法，返回一个含有分组大小的Series

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 根据与待分组的轴等长的列表或数组进行分组

In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
states

array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'], 
      dtype='|S10')

In [11]:
years = np.array([2005, 2005, 2006, 2005, 2006])
years

array([2005, 2005, 2006, 2005, 2006])

In [12]:
df['data1'].groupby([states, years]).mean() # 分组键可以是任意长度适当的数组

California  2005    0.494701
            2006   -1.082100
Ohio        2005   -0.103667
            2006   -1.171429
Name: data1, dtype: float64

### 得到分组后的内容：对分组进行迭代、字典

+ 对分组进行迭代

In [13]:
# GroupBy对象支持迭代，可以产生一组二元元组（由分组名和数据块组成），需要用for循环得到分组后的内容（与迭代器、生成器相同）
for name, group in df.groupby('key1'):
    print name
    print group

a
      data1     data2 key1 key2
0 -0.508505  0.367653    a  one
1  0.494701  0.250424    a  two
4 -1.171429  0.292727    a  one
b
     data1     data2 key1 key2
2 -1.08210 -1.702437    b  one
3  0.30117 -0.971386    b  two


In [14]:
# 对于多重键的情况，元组的第一个元素将会是由键值组成的元组
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print k1, k2
    print group

a one
      data1     data2 key1 key2
0 -0.508505  0.367653    a  one
4 -1.171429  0.292727    a  one
a two
      data1     data2 key1 key2
1  0.494701  0.250424    a  two
b one
    data1     data2 key1 key2
2 -1.0821 -1.702437    b  one
b two
     data1     data2 key1 key2
3  0.30117 -0.971386    b  two


+ 字典

In [15]:
list(df.groupby('key1'))

[('a',       data1     data2 key1 key2
  0 -0.508505  0.367653    a  one
  1  0.494701  0.250424    a  two
  4 -1.171429  0.292727    a  one), ('b',      data1     data2 key1 key2
  2 -1.08210 -1.702437    b  one
  3  0.30117 -0.971386    b  two)]

In [16]:
dict(list(df.groupby('key1')))

{'a':       data1     data2 key1 key2
 0 -0.508505  0.367653    a  one
 1  0.494701  0.250424    a  two
 4 -1.171429  0.292727    a  one, 'b':      data1     data2 key1 key2
 2 -1.08210 -1.702437    b  one
 3  0.30117 -0.971386    b  two}

In [17]:
pieces = dict(list(df.groupby('key1'))) # 将数据按分组做成一个字典，分组名：数据块
pieces['b'] # 根据分组名读取数据块

,data1,data2,key1,key2
2,-1.08210,-1.702437,b,one
3,0.30117,-0.971386,b,two


In [18]:
grouped = df.groupby(df.dtypes, axis=1) # groupby默认对行分组，通过设置axis=1可以对列分组，本例根据dtype对列进行分组
dict(list(grouped)) # 这种将数据片段做成字典的方式非常有用

{dtype('float64'):       data1     data2
 0 -0.508505  0.367653
 1  0.494701  0.250424
 2 -1.082100 -1.702437
 3  0.301170 -0.971386
 4 -1.171429  0.292727, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### 在GroupBy对象中选取一个或一组列

In [19]:
df.groupby('key1')['data1'] # 降维

In [20]:
df.groupby('key1')[['data2']] # 用一个（单个字符串）或一组（字符串数组）列名对GroupBy对象进行索引，实现选取部分列进行聚合的目的

In [21]:
df['data1'].groupby(df['key1']) # 降维

In [22]:
df[['data2']].groupby(df['key1']) # 以上两种方式等价，但是更好的办法是对整个DataFrame分组，再在GroupBy对象中索引

In [23]:
df.groupby(['key1', 'key2'])[['data2']].mean() # 传入的是列表或数组，返回已分组的DataFrame

data2 
 
 
 key1 
 key2 
 
 
 
 
 
 a 
 one 
 0.330190 
 
 
 two 
 0.250424 
 
 
 b 
 one 
 -1.702437 
 
 
 two 
 -0.971386

In [24]:
df.groupby(['key1', 'key2'])['data2'].mean() # 传入的是标量形式的单个列名，返回已分组的Series

key1  key2
a     one     0.330190
      two     0.250424
b     one    -1.702437
      two    -0.971386
Name: data2, dtype: float64

### 通过字典或Series进行分组
字典或Series，给出待分组轴上的值与分组名之间的对应关系

In [25]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan
people

,a,b,c,d,e
Joe,0.565933,0.708007,0.174525,0.599154,0.635929
Steve,0.435928,-1.094470,0.003268,-0.534766,-0.673780
Wes,1.174361,NaN,NaN,0.160553,0.621113
Jim,-1.108043,-0.839605,-0.911121,-1.356549,0.109419
Travis,-0.618798,-1.927833,-0.216986,1.426736,0.496095


In [26]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'} # 字典键是待分组DataFrame中的列名，值是分组名，顺序不可颠倒

In [27]:
by_column = people.groupby(mapping, axis=1) # 根据字典反映的分组关系对列进行分组
by_column

In [28]:
by_column.sum() # 根据分组计算列的和

,blue,red
Joe,0.773679,1.909868
Steve,-0.531499,-1.332322
Wes,0.160553,1.795475
Jim,-2.267670,-1.838229
Travis,1.209751,-2.050536


In [29]:
map_series = Series(mapping) # Series的索引是待分组DataFrame中的列名，值是分组名
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [30]:
people.groupby(map_series, axis=1).count() # 计算各分组中元素的个数，通过字典、Series分组等价

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 通过函数进行分组
任何被当做分组键的函数都会在各个**索引值**上被调用一次，其返回值就会被用作分组名称

In [31]:
people.groupby(len).sum() # 通过函数进行分组

,a,b,c,d,e
3,0.632252,-0.131599,-0.736596,-0.596842,1.366461
5,0.435928,-1.094470,0.003268,-0.534766,-0.673780
6,-0.618798,-1.927833,-0.216986,1.426736,0.496095


In [32]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min() # 函数可以跟数组、列表、字典、Series混合使用

a 
 b 
 c 
 d 
 e 
 
 
 
 
 3 
 one 
 0.565933 
 0.708007 
 0.174525 
 0.160553 
 0.621113 
 
 
 two 
 -1.108043 
 -0.839605 
 -0.911121 
 -1.356549 
 0.109419 
 
 
 5 
 one 
 0.435928 
 -1.094470 
 0.003268 
 -0.534766 
 -0.673780 
 
 
 6 
 two 
 -0.618798 
 -1.927833 
 -0.216986 
 1.426736 
 0.496095

### 根据索引级别分组
层次化索引数据集，能够根据索引级别进行聚合

In [33]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
columns

MultiIndex(levels=[[u'JP', u'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=[u'cty', u'tenor'])

In [34]:
hier_df = DataFrame(np.random.randn(4, 5), columns=columns) # 创建列层次化索引的DataFrame
hier_df

cty 
 US 
 JP 
 
 
 tenor 
 1 
 3 
 5 
 1 
 3 
 
 
 
 
 0 
 -0.066511 
 -1.596305 
 -0.156651 
 0.176348 
 -0.218685 
 
 
 1 
 -0.798463 
 1.131110 
 0.836767 
 -0.077695 
 0.477258 
 
 
 2 
 -1.216437 
 -0.646633 
 -0.862072 
 -0.148988 
 -0.018787 
 
 
 3 
 -1.314224 
 1.330123 
 0.599528 
 -1.141836 
 -1.484945

In [35]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [36]:
hier_df.groupby(level=['cty', 'tenor'], axis=1).count() # 通过level关键字传入级别编号或名称

cty 
 JP 
 US 
 
 
 tenor 
 1 
 3 
 1 
 3 
 5 
 
 
 
 
 0 
 1 
 1 
 1 
 1 
 1 
 
 
 1 
 1 
 1 
 1 
 1 
 1 
 
 
 2 
 1 
 1 
 1 
 1 
 1 
 
 
 3 
 1 
 1 
 1 
 1 
 1

In [37]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据分组聚合运算
聚合，是指任何能够从数组产生标量值的数据转换过程

In [38]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.122730,-0.570618,a,one
1,0.391450,0.446023,a,two
2,-0.684310,-0.150174,b,one
3,-0.730934,1.575972,b,two
4,0.406960,-0.592866,a,one


### 聚合运算的种类

#### 经过优化的GroupBy的方法
+ count  
分组中非NA值的数量
+ sum  
非NA值的和
+ mean  
非NA值的平均值
+ median  
非NA值的算术中位数
+ std、var  
无偏（分母为n-1）标准差和方差
+ min、max  
非NA值的最小值和最大值
+ prod  
非NA值的积
+ first、last  
第一个和最后一个非NA值

#### 分组对象上已经定义好的方法
例如quantile可以计算Series或DataFrame列的样本分位数  
原理：quantile是一个Series方法。GroupBy会高效地对Series进行切片，然后对各片调用piece.quantile(0.9)，最后将这些结果组装成最后结果

In [39]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.403858
b   -0.688972
Name: data1, dtype: float64

#### 自己定义的聚合函数
将其传入aggregate或agg方法

In [40]:
def peak_to_peak(arr):
    return arr.max() - arr.min() # 求极差
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.529690,1.038889
b,0.046624,1.726146


#### 某些非聚合运算

In [41]:
grouped.describe()

data1 
 data2 
 
 
 key1 
 
 
 
 
 
 
 
 a 
 count 
 3.000000 
 3.000000 
 
 
 mean 
 -0.108107 
 -0.239154 
 
 
 std 
 0.878724 
 0.593485 
 
 
 min 
 -1.122730 
 -0.592866 
 
 
 25% 
 -0.365640 
 -0.581742 
 
 
 50% 
 0.391450 
 -0.570618 
 
 
 75% 
 0.399205 
 -0.062297 
 
 
 max 
 0.406960 
 0.446023 
 
 
 b 
 count 
 2.000000 
 2.000000 
 
 
 mean 
 -0.707622 
 0.712899 
 
 
 std 
 0.032968 
 1.220569 
 
 
 min 
 -0.730934 
 -0.150174 
 
 
 25% 
 -0.719278 
 0.281363 
 
 
 50% 
 -0.707622 
 0.712899 
 
 
 75% 
 -0.695966 
 1.144436 
 
 
 max 
 -0.684310 
 1.575972

### 面向列的多函数应用
agg方法

In [42]:
tips = pd.read_csv('./data/ch08/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] # 添加“小费占总额百分比”的列
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


#### 对一个列应用多个函数

In [43]:
grouped = tips.groupby(['sex', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct

In [44]:
grouped_pct.agg('mean') # 对于经过优化的GroupBy的方法，可以将函数名以字符串的形式传入

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [45]:
grouped_pct.agg(['mean', 'std', peak_to_peak]) # 对分组应用多个函数，传入一组函数或函数名，得到的DataFrame的列会以相应的函数命名

mean 
 std 
 peak_to_peak 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 Female 
 No 
 0.156921 
 0.036421 
 0.195876 
 
 
 Yes 
 0.182150 
 0.071595 
 0.360233 
 
 
 Male 
 No 
 0.160669 
 0.041849 
 0.220186 
 
 
 Yes 
 0.152771 
 0.090588 
 0.674707

#### 为返回的DataFrame的列命名

In [46]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]) # 传入(name, function)元组组成的列表，各元组的第一个元素会被用作返回的DataFrame的列名

foo 
 bar 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 Female 
 No 
 0.156921 
 0.036421 
 
 
 Yes 
 0.182150 
 0.071595 
 
 
 Male 
 No 
 0.160669 
 0.041849 
 
 
 Yes 
 0.152771 
 0.090588

#### 对多个列应用多个函数

In [47]:
functions = ['count', 'mean', 'max'] # 定义一组应用于全部列的函数
result = grouped['tip_pct', 'total_bill'].agg(functions) # 对多个列的分组应用事先定义的一组函数
result

tip_pct 
 total_bill 
 
 
 
 
 count 
 mean 
 max 
 count 
 mean 
 max 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 
 
 
 Female 
 No 
 54 
 0.156921 
 0.252672 
 54 
 18.105185 
 35.83 
 
 
 Yes 
 33 
 0.182150 
 0.416667 
 33 
 17.977879 
 44.30 
 
 
 Male 
 No 
 97 
 0.160669 
 0.291990 
 97 
 19.791237 
 48.33 
 
 
 Yes 
 60 
 0.152771 
 0.710345 
 60 
 22.284500 
 50.81

In [48]:
result['tip_pct'] # 返回的DataFrame拥有层次化的列，相当于分别对各列进行聚合，然后用concat将结果组装到一起（列名用作keys参数）

count 
 mean 
 max 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 Female 
 No 
 54 
 0.156921 
 0.252672 
 
 
 Yes 
 33 
 0.182150 
 0.416667 
 
 
 Male 
 No 
 97 
 0.160669 
 0.291990 
 
 
 Yes 
 60 
 0.152771 
 0.710345

In [49]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)] # 传入带有自定义名称的元组列表
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct 
 total_bill 
 
 
 
 
 Durchschnitt 
 Abweichung 
 Durchschnitt 
 Abweichung 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 
 Female 
 No 
 0.156921 
 0.001327 
 18.105185 
 53.092422 
 
 
 Yes 
 0.182150 
 0.005126 
 17.977879 
 84.451517 
 
 
 Male 
 No 
 0.160669 
 0.001751 
 19.791237 
 76.152961 
 
 
 Yes 
 0.152771 
 0.008206 
 22.284500 
 98.244673

#### 对不同的列应用不同的函数
向agg传入一个从列名映射到函数的字典

In [50]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip 
 size 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 Female 
 No 
 5.2 
 140 
 
 
 Yes 
 6.5 
 74 
 
 
 Male 
 No 
 9.0 
 263 
 
 
 Yes 
 10.0 
 150

In [51]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct 
 size 
 
 
 
 
 min 
 max 
 mean 
 std 
 sum 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 
 
 Female 
 No 
 0.056797 
 0.252672 
 0.156921 
 0.036421 
 140 
 
 
 Yes 
 0.056433 
 0.416667 
 0.182150 
 0.071595 
 74 
 
 
 Male 
 No 
 0.071804 
 0.291990 
 0.160669 
 0.041849 
 263 
 
 
 Yes 
 0.035638 
 0.710345 
 0.152771 
 0.090588 
 150

### 以“无索引”的形式返回聚合数据
返回的DataFrame中，分组键默认作为索引，向groupby传入as_index=False可以禁用该功能

In [52]:
tips.groupby(['sex', 'smoker'], as_index=False).mean() # 分组键不再是返回的DataFrame的索引，而是列

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


In [53]:
tips.groupby(['sex', 'smoker']).mean().reset_index() # 以上两种方式等价

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


## 分组级运算和转换

In [54]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.974984,-1.458483,a,one
1,-2.029525,-1.573788,a,two
2,1.075888,-1.801839,b,one
3,0.819511,0.183579,b,two
4,1.332829,1.990050,a,one


### 先聚合再合并

In [55]:
k1_means = df.groupby('key1').mean().add_prefix('mean_') # # df.add_prefix()，给df的所有列名添加前缀；同理还有df.add_suffix()
k1_means

,mean_data1,mean_data2
key1,,
a,0.426096,-0.347407
b,0.947700,-0.809130


In [56]:
pd.merge(df, k1_means, left_on='key1', right_index=True) # 先聚合再合并

,data1,data2,key1,key2,mean_data1,mean_data2
0,1.974984,-1.458483,a,one,0.426096,-0.347407
1,-2.029525,-1.573788,a,two,0.426096,-0.347407
4,1.332829,1.990050,a,one,0.426096,-0.347407
2,1.075888,-1.801839,b,one,0.947700,-0.809130
3,0.819511,0.183579,b,two,0.947700,-0.809130


### transform方法
要求：传入的函数要么产生一个可以广播的标量值，要么产生一个相同大小的结果数组

In [57]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,-0.875233,0.938646,1.032062,0.446559,-1.392066
Steve,1.374844,-1.501469,-0.853110,0.943532,-1.578382
Wes,-0.318920,-0.939677,1.100827,-0.345144,0.400290
Jim,1.283040,-1.083932,-0.387142,-0.773780,-1.642887
Travis,0.681558,0.572325,-0.061930,-0.625450,1.353195


In [58]:
key = ['one', 'two', 'one', 'two', 'one']
people.groupby(key).mean()

,a,b,c,d,e
one,-0.170865,0.190431,0.690320,-0.174678,0.120473
two,1.328942,-1.292700,-0.620126,0.084876,-1.610634


In [59]:
people.groupby(key).transform(np.mean) # transform将一个函数应用到各个分组，然后将结果放置到适当的位置上，如果各分组产生标量值，则广播出去

,a,b,c,d,e
Joe,-0.170865,0.190431,0.690320,-0.174678,0.120473
Steve,1.328942,-1.292700,-0.620126,0.084876,-1.610634
Wes,-0.170865,0.190431,0.690320,-0.174678,0.120473
Jim,1.328942,-1.292700,-0.620126,0.084876,-1.610634
Travis,-0.170865,0.190431,0.690320,-0.174678,0.120473


In [60]:
def demean(arr):
    return arr - arr.mean() # 计算离差
demeaned = people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,-0.704368,0.748215,0.341742,0.621237,-1.512539
Steve,0.045902,-0.208768,-0.232984,0.858656,0.032253
Wes,-0.148055,-1.130108,0.410508,-0.170465,0.279817
Jim,-0.045902,0.208768,0.232984,-0.858656,-0.032253
Travis,0.852423,0.381893,-0.752250,-0.450772,1.232722


In [61]:
demeaned.groupby(key).mean() # 检查各分组离差的均值是否为0

,a,b,c,d,e
one,0.0,0.000000e+00,0.000000e+00,-3.700743e-17,0.000000e+00
two,0.0,-1.110223e-16,-5.551115e-17,0.000000e+00,-1.110223e-16


### apply: 一般性的“拆分-应用-合并”
相比于transform方法，apply方法更加一般化，只要传入的函数能够产生一个pandas对象或标量值即可。
apply方法会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试将各片段组合到一起。

In [62]:
tips = pd.read_csv('./data/ch08/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] # 添加“小费占总额百分比”的列
tips[:10]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240
6,8.77,2.00,Male,No,Sun,Dinner,2,0.228050
7,26.88,3.12,Male,No,Sun,Dinner,4,0.116071
8,15.04,1.96,Male,No,Sun,Dinner,2,0.130319
9,14.78,3.23,Male,No,Sun,Dinner,2,0.218539


In [63]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:] #　编写函数，在df中的指定列找出最大的5个值，并在值所在的行选取出来
top(tips, n=6) # 在tips中的'tip_pct'列找出最大的6个值，并在值所在的行选取出来

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [64]:
tips.groupby('smoker').apply(top) # top函数在各个分组片段上调用，结果由pd.concat组装，并以分组名称进行标记，形成层次化索引

total_bill 
 tip 
 sex 
 smoker 
 day 
 time 
 size 
 tip_pct 
 
 
 smoker 
 
 
 
 
 
 
 
 
 
 
 
 
 
 No 
 88 
 24.71 
 5.85 
 Male 
 No 
 Thur 
 Lunch 
 2 
 0.236746 
 
 
 185 
 20.69 
 5.00 
 Male 
 No 
 Sun 
 Dinner 
 5 
 0.241663 
 
 
 51 
 10.29 
 2.60 
 Female 
 No 
 Sun 
 Dinner 
 2 
 0.252672 
 
 
 149 
 7.51 
 2.00 
 Male 
 No 
 Thur 
 Lunch 
 2 
 0.266312 
 
 
 232 
 11.61 
 3.39 
 Male 
 No 
 Sat 
 Dinner 
 2 
 0.291990 
 
 
 Yes 
 109 
 14.31 
 4.00 
 Female 
 Yes 
 Sat 
 Dinner 
 2 
 0.279525 
 
 
 183 
 23.17 
 6.50 
 Male 
 Yes 
 Sun 
 Dinner 
 4 
 0.280535 
 
 
 67 
 3.07 
 1.00 
 Female 
 Yes 
 Sat 
 Dinner 
 1 
 0.325733 
 
 
 178 
 9.60 
 4.00 
 Female 
 Yes 
 Sun 
 Dinner 
 2 
 0.416667 
 
 
 172 
 7.25 
 5.15 
 Male 
 Yes 
 Sun 
 Dinner 
 2 
 0.710345

In [65]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill') # 如果传给apply的函数能够接受其他参数或关键字，则放在函数名后一并传入

total_bill 
 tip 
 sex 
 smoker 
 day 
 time 
 size 
 tip_pct 
 
 
 smoker 
 day 
 
 
 
 
 
 
 
 
 
 
 
 
 
 No 
 Fri 
 94 
 22.75 
 3.25 
 Female 
 No 
 Fri 
 Dinner 
 2 
 0.142857 
 
 
 Sat 
 212 
 48.33 
 9.00 
 Male 
 No 
 Sat 
 Dinner 
 4 
 0.186220 
 
 
 Sun 
 156 
 48.17 
 5.00 
 Male 
 No 
 Sun 
 Dinner 
 6 
 0.103799 
 
 
 Thur 
 142 
 41.19 
 5.00 
 Male 
 No 
 Thur 
 Lunch 
 5 
 0.121389 
 
 
 Yes 
 Fri 
 95 
 40.17 
 4.73 
 Male 
 Yes 
 Fri 
 Dinner 
 4 
 0.117750 
 
 
 Sat 
 170 
 50.81 
 10.00 
 Male 
 Yes 
 Sat 
 Dinner 
 3 
 0.196812 
 
 
 Sun 
 182 
 45.35 
 3.50 
 Male 
 Yes 
 Sun 
 Dinner 
 3 
 0.077178 
 
 
 Thur 
 197 
 43.11 
 5.00 
 Female 
 Yes 
 Thur 
 Lunch 
 4 
 0.115982

In [66]:
result = tips.groupby('smoker')['tip_pct'].describe() # 在GroupBy对象上调用describe方法
result

smoker       
No      count    151.000000
        mean       0.159328
        std        0.039910
        min        0.056797
        25%        0.136906
        50%        0.155625
        75%        0.185014
        max        0.291990
Yes     count     93.000000
        mean       0.163196
        std        0.085119
        min        0.035638
        25%        0.106771
        50%        0.153846
        75%        0.195059
        max        0.710345
Name: tip_pct, dtype: float64

In [67]:
result.unstack('smoker')

smoker,No,Yes
count,151.000000,93.000000
mean,0.159328,0.163196
std,0.039910,0.085119
min,0.056797,0.035638
25%,0.136906,0.106771
50%,0.155625,0.153846
75%,0.185014,0.195059
max,0.291990,0.710345


In [68]:
f = lambda x: x.describe()
grouped.apply(f) # 在GroupBy中，调用诸如describe之类的方法时，实际上只是应用了这两天代码的快捷方式

total_bill 
 tip 
 size 
 tip_pct 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 
 
 Female 
 No 
 count 
 54.000000 
 54.000000 
 54.000000 
 54.000000 
 
 
 mean 
 18.105185 
 2.773519 
 2.592593 
 0.156921 
 
 
 std 
 7.286455 
 1.128425 
 1.073146 
 0.036421 
 
 
 min 
 7.250000 
 1.000000 
 1.000000 
 0.056797 
 
 
 25% 
 12.650000 
 2.000000 
 2.000000 
 0.139708 
 
 
 50% 
 16.690000 
 2.680000 
 2.000000 
 0.149691 
 
 
 75% 
 20.862500 
 3.437500 
 3.000000 
 0.181630 
 
 
 max 
 35.830000 
 5.200000 
 6.000000 
 0.252672 
 
 
 Yes 
 count 
 33.000000 
 33.000000 
 33.000000 
 33.000000 
 
 
 mean 
 17.977879 
 2.931515 
 2.242424 
 0.182150 
 
 
 std 
 9.189751 
 1.219916 
 0.613917 
 0.071595 
 
 
 min 
 3.070000 
 1.000000 
 1.000000 
 0.056433 
 
 
 25% 
 12.760000 
 2.000000 
 2.000000 
 0.152439 
 
 
 50% 
 16.270000 
 2.880000 
 2.000000 
 0.173913 
 
 
 75% 
 22.120000 
 3.500000 
 2.000000 
 0.198216 
 
 
 max 
 44.300000 
 6.500000 
 4.000000 
 0.416667 
 
 
 Male 
 No 
 count 
 97.000000 
 97.000000 
 97.000000 
 97.000000 
 
 
 mean 
 19.791237 
 3.113402 
 2.711340 
 0.160669 
 
 
 std 
 8.726566 
 1.489559 
 0.989094 
 0.041849 
 
 
 min 
 7.510000 
 1.250000 
 2.000000 
 0.071804 
 
 
 25% 
 13.810000 
 2.000000 
 2.000000 
 0.131810 
 
 
 50% 
 18.240000 
 2.740000 
 2.000000 
 0.157604 
 
 
 75% 
 22.820000 
 3.710000 
 3.000000 
 0.186220 
 
 
 max 
 48.330000 
 9.000000 
 6.000000 
 0.291990 
 
 
 Yes 
 count 
 60.000000 
 60.000000 
 60.000000 
 60.000000 
 
 
 mean 
 22.284500 
 3.051167 
 2.500000 
 0.152771 
 
 
 std 
 9.911845 
 1.500120 
 0.892530 
 0.090588 
 
 
 min 
 7.250000 
 1.000000 
 1.000000 
 0.035638 
 
 
 25% 
 15.272500 
 2.000000 
 2.000000 
 0.101845 
 
 
 50% 
 20.390000 
 3.000000 
 2.000000 
 0.141015 
 
 
 75% 
 28.572500 
 3.820000 
 3.000000 
 0.191697 
 
 
 max 
 50.810000 
 10.000000 
 5.000000 
 0.710345

#### 禁止分组键

In [69]:
tips.groupby('smoker', group_keys=False).apply(top) # 禁止分组名称形成层次化索引，相当于后面加了个reset_index方法

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析
将cut、qcut等函数与groupby函数结合起来，以实现对数据集的桶（bucket）-cut函数或分位数（quantile）-qcut函数分析

#### 桶分析

In [70]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})
frame[:10]

,data1,data2
0,-0.313630,1.040411
1,-0.449557,-1.943923
2,-1.156233,-1.964678
3,1.242925,-0.553908
4,0.459095,-1.004519
5,-0.440166,0.444652
6,2.218262,-0.026778
7,-1.386905,-0.640124
8,-1.339748,-0.415983
9,1.062130,1.708168


In [71]:
factor = pd.cut(frame.data1, 4)
factor[:10]

0    (-1.735, -0.0811]
1    (-1.735, -0.0811]
2    (-1.735, -0.0811]
3     (-0.0811, 1.573]
4     (-0.0811, 1.573]
5    (-1.735, -0.0811]
6       (1.573, 3.227]
7    (-1.735, -0.0811]
8    (-1.735, -0.0811]
9     (-0.0811, 1.573]
Name: data1, dtype: category
Categories (4, object): [(-3.396, -1.735] < (-1.735, -0.0811] < (-0.0811, 1.573] < (1.573, 3.227]]

In [72]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(factor) # 由cut返回的对象可直接用于groupby，相当于根据data1的桶，计算data2的统计量
grouped.apply(get_stats).unstack() # 将层次化索引转换为更直观的表形式

,count,max,mean,min
data1,,,,
"(-3.396, -1.735]",42.0,2.251162,-0.079456,-2.576339
"(-1.735, -0.0811]",426.0,3.406842,0.017318,-2.455777
"(-0.0811, 1.573]",476.0,3.017645,-0.015145,-3.084366
"(1.573, 3.227]",56.0,2.298403,-0.084767,-2.981965


#### 分位数

In [73]:
grouping = pd.qcut(frame.data1, 10, labels=False) # 传入labels=False即可只获取分位数的编号，即只显示levels

grouped = frame.data2.groupby(grouping) # 由qcut返回的对象可直接用于groupby，相当于根据data1的分位数，计算data2的统计量
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.436572,-0.094053,-2.576339
1,100.0,2.611112,-0.199330,-2.006313
2,100.0,3.406842,0.224177,-2.246100
3,100.0,3.252699,0.008798,-2.260766
4,100.0,2.874288,0.142576,-2.455777
5,100.0,2.404728,0.024585,-2.482092
6,100.0,3.017645,-0.038479,-2.674240
7,100.0,2.114260,-0.100899,-2.231351
8,100.0,2.389221,-0.003837,-3.084366


## 透视表和交叉表

### 透视表
透视表（pivot table）是各种电子表格程序和其他数据分析软件中一种常见的数据汇总工具。它根据一个或多个键对数据进行聚合，并根据行和列上的分组键将数据分配到各个矩形区域中
+ values  
待聚合的列的名称。默认聚合所有数值列
+ index  
用于分组的列名或其他分组键，出现在结果透视表的行
+ columns  
用于分组的列名或其他分组键，出现在结果透视表的列
+ aggfunc  
聚合函数或聚合的列表，默认为'mean'。可以是任何对groupby有效的函数
+ fill_value  
用于替换结果表中的缺失值
+ margins  
添加行/列小计和总计，默认为False

In [74]:
tips = pd.read_csv('./data/ch08/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] # 添加“小费占总额百分比”的列
tips[:10]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240
6,8.77,2.00,Male,No,Sun,Dinner,2,0.228050
7,26.88,3.12,Male,No,Sun,Dinner,4,0.116071
8,15.04,1.96,Male,No,Sun,Dinner,2,0.130319
9,14.78,3.23,Male,No,Sun,Dinner,2,0.218539


In [75]:
tips.pivot_table(index=['sex', 'smoker']) # 根据sex和smoker计算分组平均数（pivot_table的默认聚合类型）

size 
 tip 
 tip_pct 
 total_bill 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 
 Female 
 No 
 2.592593 
 2.773519 
 0.156921 
 18.105185 
 
 
 Yes 
 2.242424 
 2.931515 
 0.182150 
 17.977879 
 
 
 Male 
 No 
 2.711340 
 3.113402 
 0.160669 
 19.791237 
 
 
 Yes 
 2.500000 
 3.051167 
 0.152771 
 22.284500

In [76]:
tips.groupby(['sex', 'smoker']).mean() # 以上两种方式等价

total_bill 
 tip 
 size 
 tip_pct 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 
 Female 
 No 
 18.105185 
 2.773519 
 2.592593 
 0.156921 
 
 
 Yes 
 17.977879 
 2.931515 
 2.242424 
 0.182150 
 
 
 Male 
 No 
 19.791237 
 3.113402 
 2.711340 
 0.160669 
 
 
 Yes 
 22.284500 
 3.051167 
 2.500000 
 0.152771

In [77]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],
                 columns='smoker') # 聚合'tip_pct'和'size'列，行上根据'sex'和'day'分组，列上根据'day'分组

tip_pct 
 size 
 
 
 
 smoker 
 No 
 Yes 
 No 
 Yes 
 
 
 sex 
 day 
 
 
 
 
 
 
 
 
 Female 
 Fri 
 0.165296 
 0.209129 
 2.500000 
 2.000000 
 
 
 Sat 
 0.147993 
 0.163817 
 2.307692 
 2.200000 
 
 
 Sun 
 0.165710 
 0.237075 
 3.071429 
 2.500000 
 
 
 Thur 
 0.155971 
 0.163073 
 2.480000 
 2.428571 
 
 
 Male 
 Fri 
 0.138005 
 0.144730 
 2.000000 
 2.125000 
 
 
 Sat 
 0.162132 
 0.139067 
 2.656250 
 2.629630 
 
 
 Sun 
 0.158291 
 0.173964 
 2.883721 
 2.600000 
 
 
 Thur 
 0.165706 
 0.164417 
 2.500000 
 2.300000

In [78]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],
                 columns='smoker', margins=True) # 传入margins=True添加分项小计，添加标签为All的列和行，值对应单个等级中所有数据的分组统计

tip_pct 
 size 
 
 
 
 smoker 
 No 
 Yes 
 All 
 No 
 Yes 
 All 
 
 
 sex 
 day 
 
 
 
 
 
 
 
 
 
 
 Female 
 Fri 
 0.165296 
 0.209129 
 0.199388 
 2.500000 
 2.000000 
 2.111111 
 
 
 Sat 
 0.147993 
 0.163817 
 0.156470 
 2.307692 
 2.200000 
 2.250000 
 
 
 Sun 
 0.165710 
 0.237075 
 0.181569 
 3.071429 
 2.500000 
 2.944444 
 
 
 Thur 
 0.155971 
 0.163073 
 0.157525 
 2.480000 
 2.428571 
 2.468750 
 
 
 Male 
 Fri 
 0.138005 
 0.144730 
 0.143385 
 2.000000 
 2.125000 
 2.100000 
 
 
 Sat 
 0.162132 
 0.139067 
 0.151577 
 2.656250 
 2.629630 
 2.644068 
 
 
 Sun 
 0.158291 
 0.173964 
 0.162344 
 2.883721 
 2.600000 
 2.810345 
 
 
 Thur 
 0.165706 
 0.164417 
 0.165276 
 2.500000 
 2.300000 
 2.433333 
 
 
 All 
 
 0.159328 
 0.163196 
 0.160803 
 2.668874 
 2.408602 
 2.569672

In [79]:
tips.pivot_table('tip_pct', index=['sex', 'smoker'], columns='day',
                 aggfunc=len, margins=True) # 使用其他聚合函数，将其传给aggfunc即可；使用len可以得到有关分组大小的交叉表

day 
 Fri 
 Sat 
 Sun 
 Thur 
 All 
 
 
 sex 
 smoker 
 
 
 
 
 
 
 
 
 
 Female 
 No 
 2.0 
 13.0 
 14.0 
 25.0 
 54.0 
 
 
 Yes 
 7.0 
 15.0 
 4.0 
 7.0 
 33.0 
 
 
 Male 
 No 
 2.0 
 32.0 
 43.0 
 20.0 
 97.0 
 
 
 Yes 
 8.0 
 27.0 
 15.0 
 10.0 
 60.0 
 
 
 All 
 
 19.0 
 87.0 
 76.0 
 62.0 
 244.0

In [80]:
tips.pivot_table('size', index=['time', 'sex', 'smoker'],
                 columns='day', aggfunc='sum', fill_value=0) # 如果存在空的组合（NA），可以设置fill_value

day 
 Fri 
 Sat 
 Sun 
 Thur 
 
 
 time 
 sex 
 smoker 
 
 
 
 
 
 
 
 
 Dinner 
 Female 
 No 
 2 
 30 
 43 
 2 
 
 
 Yes 
 8 
 33 
 10 
 0 
 
 
 Male 
 No 
 4 
 85 
 124 
 0 
 
 
 Yes 
 12 
 71 
 39 
 0 
 
 
 Lunch 
 Female 
 No 
 3 
 0 
 0 
 60 
 
 
 Yes 
 6 
 0 
 0 
 17 
 
 
 Male 
 No 
 0 
 0 
 0 
 50 
 
 
 Yes 
 5 
 0 
 0 
 23

### 交叉表: crosstab
交叉表（cross-tabulation，简称crosstab）是一种用于计算**分组频率**的特殊透视表

In [81]:
from StringIO import StringIO
data = """\
Sample    Gender    Handedness
1    Female    Right-handed
2    Male    Left-handed
3    Female    Right-handed
4    Male    Right-handed
5    Male    Left-handed
6    Male    Right-handed
7    Female    Right-handed
8    Female    Left-handed
9    Male    Right-handed
10    Female    Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')
data

,Sample,Gender,Handedness
0,1,Female,Right-handed
1,2,Male,Left-handed
2,3,Female,Right-handed
3,4,Male,Right-handed
4,5,Male,Left-handed
5,6,Male,Right-handed
6,7,Female,Right-handed
7,8,Female,Left-handed
8,9,Male,Right-handed
9,10,Female,Right-handed


In [82]:
pd.crosstab(data['Gender'], data['Handedness'], margins=True) # 根据性别和用手习惯对数据集进行汇总统计

Handedness,Left-handed,Right-handed,All
Gender,,,
Female,1,4,5
Male,2,3,5
All,3,7,10


In [83]:
data.pivot_table(index='Gender', columns='Handedness', aggfunc=len, margins=True) # 以上两种方式等价

Sample 
 
 
 Handedness 
 Left-handed 
 Right-handed 
 All 
 
 
 Gender 
 
 
 
 
 
 
 
 Female 
 1.0 
 4.0 
 5.0 
 
 
 Male 
 2.0 
 3.0 
 5.0 
 
 
 All 
 3.0 
 7.0 
 10.0

In [84]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True) # crosstab的前两个参数可以是数组、Series或数组列表

smoker 
 No 
 Yes 
 All 
 
 
 time 
 day 
 
 
 
 
 
 
 
 Dinner 
 Fri 
 3 
 9 
 12 
 
 
 Sat 
 45 
 42 
 87 
 
 
 Sun 
 57 
 19 
 76 
 
 
 Thur 
 1 
 0 
 1 
 
 
 Lunch 
 Fri 
 1 
 6 
 7 
 
 
 Thur 
 44 
 17 
 61 
 
 
 All 
 
 151 
 93 
 244

### Example: Filling missing values with group-specific values

In [ ]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s

In [ ]:
s.fillna(s.mean())

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

In [ ]:
data.groupby(group_key).mean()

In [ ]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

In [ ]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

### Example: Random sampling and permutation

In [ ]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (range(1, 11) + [10] * 3) * 4
base_names = ['A'] + range(2, 11) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = Series(card_val, index=cards)

In [ ]:
deck[:13]

In [ ]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

In [ ]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
# alternatively
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

### Example: Group weighted average and correlation

In [ ]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})
df

In [ ]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

In [ ]:
close_px = pd.read_csv('ch09/stock_px.csv', parse_dates=True, index_col=0)
close_px.info()

In [ ]:
close_px[-4:]

In [ ]:
rets = close_px.pct_change().dropna()
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x: x.year)
by_year.apply(spx_corr)

In [ ]:
# Annual correlation of Apple with Microsoft
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### Example: Group-wise linear regression

In [ ]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
by_year.apply(regress, 'AAPL', ['SPX'])

## Example: 2012 Federal Election Commission Database

In [ ]:
fec = pd.read_csv('ch09/P00000001-ALL.csv')

In [ ]:
fec.info()

In [ ]:
fec.ix[123456]

In [ ]:
unique_cands = fec.cand_nm.unique()
unique_cands

In [ ]:
unique_cands[2]

In [ ]:
parties = {'Bachmann, Michelle': 'Republican',
           'Cain, Herman': 'Republican',
           'Gingrich, Newt': 'Republican',
           'Huntsman, Jon': 'Republican',
           'Johnson, Gary Earl': 'Republican',
           'McCotter, Thaddeus G': 'Republican',
           'Obama, Barack': 'Democrat',
           'Paul, Ron': 'Republican',
           'Pawlenty, Timothy': 'Republican',
           'Perry, Rick': 'Republican',
           "Roemer, Charles E. 'Buddy' III": 'Republican',
           'Romney, Mitt': 'Republican',
           'Santorum, Rick': 'Republican'}

In [ ]:
fec.cand_nm[123456:123461]

In [ ]:
fec.cand_nm[123456:123461].map(parties)

In [ ]:
# Add it as a column
fec['party'] = fec.cand_nm.map(parties)

In [ ]:
fec['party'].value_counts()

In [ ]:
(fec.contb_receipt_amt > 0).value_counts()

In [ ]:
fec = fec[fec.contb_receipt_amt > 0]

In [ ]:
fec_mrbo = fec[fec.cand_nm.isin(['Obama, Barack', 'Romney, Mitt'])]

### Donation statistics by occupation and employer

In [ ]:
fec.contbr_occupation.value_counts()[:10]

In [ ]:
occ_mapping = {
   'INFORMATION REQUESTED PER BEST EFFORTS' : 'NOT PROVIDED',
   'INFORMATION REQUESTED' : 'NOT PROVIDED',
   'INFORMATION REQUESTED (BEST EFFORTS)' : 'NOT PROVIDED',
   'C.E.O.': 'CEO'
}

# If no mapping provided, return x
f = lambda x: occ_mapping.get(x, x)
fec.contbr_occupation = fec.contbr_occupation.map(f)

In [ ]:
emp_mapping = {
   'INFORMATION REQUESTED PER BEST EFFORTS' : 'NOT PROVIDED',
   'INFORMATION REQUESTED' : 'NOT PROVIDED',
   'SELF' : 'SELF-EMPLOYED',
   'SELF EMPLOYED' : 'SELF-EMPLOYED',
}

# If no mapping provided, return x
f = lambda x: emp_mapping.get(x, x)
fec.contbr_employer = fec.contbr_employer.map(f)

In [ ]:
by_occupation = fec.pivot_table('contb_receipt_amt',
                                index='contbr_occupation',
                                columns='party', aggfunc='sum')

In [ ]:
over_2mm = by_occupation[by_occupation.sum(1) > 2000000]
over_2mm

In [ ]:
over_2mm.plot(kind='barh')

In [ ]:
def get_top_amounts(group, key, n=5):
    totals = group.groupby(key)['contb_receipt_amt'].sum()

    # Order totals by key in descending order
    return totals.sort_values(ascending=False)[-n:]

In [ ]:
grouped = fec_mrbo.groupby('cand_nm')
grouped.apply(get_top_amounts, 'contbr_occupation', n=7)

In [ ]:
grouped.apply(get_top_amounts, 'contbr_employer', n=10)

### Bucketing donation amounts

In [ ]:
bins = np.array([0, 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000])
labels = pd.cut(fec_mrbo.contb_receipt_amt, bins)
labels

In [ ]:
grouped = fec_mrbo.groupby(['cand_nm', labels])
grouped.size().unstack(0)

In [ ]:
bucket_sums = grouped.contb_receipt_amt.sum().unstack(0)
bucket_sums

In [ ]:
normed_sums = bucket_sums.div(bucket_sums.sum(axis=1), axis=0)
normed_sums

In [ ]:
normed_sums[:-2].plot(kind='barh', stacked=True)

### Donation statistics by state

In [ ]:
grouped = fec_mrbo.groupby(['cand_nm', 'contbr_st'])
totals = grouped.contb_receipt_amt.sum().unstack(0).fillna(0)
totals = totals[totals.sum(1) > 100000]
totals[:10]

In [ ]:
percent = totals.div(totals.sum(1), axis=0)
percent[:10]